In [1]:
import numpy as np

import os
import glob #tqdm
from sklearn import preprocessing
from sklearn.preprocessing import*
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tensorflow.keras.utils import to_categorical


#Neural Network
import tensorflow
from tensorflow.keras.layers import Conv2D,Dense,AveragePooling2D, Flatten
from tensorflow.keras.optimizers     import SGD
from tensorflow.keras.models import Sequential, load_model

#visualization
import matplotlib.pyplot as plt
%matplotlib inline

#OpenCv
import cv2


#warning
import warnings
warnings.filterwarnings('ignore')


C:\Users\Anik\anaconda3\envs\Deep\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
testimageDatasetspath = "E:/DeepLearning/Datasets/train" 
datasetsNameTest = os.listdir(testimageDatasetspath)

In [3]:
a = list(np.random.randint(0,100,size=20))
a

[55, 71, 40, 27, 34, 51, 60, 67, 94, 15, 82, 79, 4, 66, 27, 79, 79, 11, 74, 87]

In [4]:
np.unique(datasetsNameTest)

array(['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', ..., 'dog.998.jpg',
       'dog.999.jpg', 'dogs'], dtype='<U11')

In [5]:
datasetsNameTest.index("dog.999.jpg")

2000

In [6]:
image_width = 28
image_height = 28

In [7]:
def imageFeatureExtract(image,size =(28,28)):
    img = cv2.resize(image,size)
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    gray = np.expand_dims(gray,2) #(28,28,1)
    return gray

In [8]:
Datasetspath = "E:/DeepLearning/Datasets/train"
datasetPath = os.path.join(Datasetspath , "*g")
fileRead = glob.glob(datasetPath)

data =[]
category_or_class = []

for (i,file) in enumerate(fileRead):
    image = cv2.imread(file)
    feature = imageFeatureExtract(image)
    data.append(feature)
    classNames = file.split(os.path.sep)[-1].split(".")[0]
    category_or_class.append(classNames)

In [9]:
## Datasets Scalling
data = np.array(data, dtype= np.uint8)
labels = np.array(category_or_class)

In [10]:
data.shape

(2000, 28, 28, 1)

In [11]:
np.unique(category_or_class)

array(['cat', 'dog'], dtype='<U3')

In [12]:
len(category_or_class)

2000

In [13]:
type(category_or_class)

list

## Step 1:Label Encoder


In [14]:
le = preprocessing.LabelEncoder()
le.fit(category_or_class)
list(le.classes_)
classNames = le.transform(category_or_class)

In [15]:
np.unique(classNames)


array([0, 1])

## Step 2: OneHotEnconding


In [16]:
oneHotEncoder = OneHotEncoder(sparse= False)
label_EncoderValue  = classNames.reshape(len(classNames), 1)
oneHotEncoderValues = oneHotEncoder.fit_transform(label_EncoderValue)

In [17]:
oneHotEncoderValues


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [18]:
(X_train,X_test, Y_train,Y_test) = train_test_split(data, oneHotEncoderValues, test_size=0.2 ,random_state=42)

In [19]:
model = Sequential()
 #1st layer
model.add(Conv2D(6, kernel_size = (5,5) , strides = (1,1) , activation ="tanh", input_shape =(28,28,1),  padding ="same"))
model.add(AveragePooling2D(pool_size  = (2,2),strides=(2,2),padding="valid")) 
  
 #2nd layer
model.add(Conv2D(16, kernel_size = (5,5) ,strides = (1,1), activation ="tanh", input_shape =(28,28,1), padding ="valid"))
model.add(AveragePooling2D(pool_size  = (2,2),strides=(2,2),padding="valid")) 
          
   #flatten layer
model.add(Flatten())
          
  #output layer 
model.add(Dense(120, activation= "tanh"))
model.add(Dense(84, activation= "tanh"))          
model.add(Dense(2, activation= "softmax"))          
          
          
          

In [20]:
model.compile(optimizer= "adam",
             loss="categorical_crossentropy",
             metrics=["accuracy"])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 120)               4

In [22]:
model.fit(X_train,Y_train,
          epochs =10,
          validation_data=(X_test,Y_test),verbose=1)

Epoch 1/10
50/50 [==============================] - 3s 14ms/step - loss: 0.7516 - accuracy: 0.5119 - val_loss: 0.6927 - val_accuracy: 0.5050
Epoch 2/10
50/50 [==============================] - 0s 8ms/step - loss: 0.6781 - accuracy: 0.5781 - val_loss: 0.6656 - val_accuracy: 0.6000
Epoch 3/10
50/50 [==============================] - 0s 8ms/step - loss: 0.6604 - accuracy: 0.6025 - val_loss: 0.6914 - val_accuracy: 0.6100
Epoch 4/10
50/50 [==============================] - 0s 7ms/step - loss: 0.6326 - accuracy: 0.6375 - val_loss: 0.6882 - val_accuracy: 0.5750
Epoch 5/10
50/50 [==============================] - 0s 8ms/step - loss: 0.6412 - accuracy: 0.6413 - val_loss: 0.6806 - val_accuracy: 0.5950
Epoch 6/10
50/50 [==============================] - 0s 7ms/step - loss: 0.6087 - accuracy: 0.6719 - val_loss: 0.6999 - val_accuracy: 0.5675
Epoch 7/10
50/50 [==============================] - 0s 8ms/step - loss: 0.6245 - accuracy: 0.6525 - val_loss: 0.6741 - val_accuracy: 0.6000
Epoch 8/10
50/50 [=

In [23]:
class LeNet_Model(Sequential):
    
    def __init__(self,input_shape,number_class):
        super().__init__()
        
         #1st layer
        self.add(Conv2D(6, kernel_size = (5,5) , strides = (1,1) , activation ="tanh", input_shape =input_shape,padding="same"))
        self.add(AveragePooling2D(pool_size  = (2,2),strides=(2,2),padding="valid"))
        
        #2nd layer
        self.add(Conv2D(16,kernel_size = (5,5) , strides = (1,1) , activation ="tanh",   padding ="valid"))
        self.add(AveragePooling2D(pool_size  = (2,2),strides=(2,2),padding="valid"))
        
        self.add(Flatten())
        
        self.add(Dense(120, activation= "tanh"))
        self.add(Dense(84,activation="tanh"))
        self.add(Dense(number_class , activation="softmax"))
        
        self.compile(optimizer= "adam",
             loss="categorical_crossentropy",
             metrics=["accuracy"])
        


In [24]:
input_shapes = X_train[0].shape
number_class = len(np.unique(labels))

In [25]:
myModel =LeNet_Model(input_shapes,number_class)
myModel.summary()

Model: "le_net__model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_2 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_3 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 400)               0         
                                                                 
 dense_3 (Dense)             (None, 120)             

In [26]:
import datetime 
import tensorflow as tf
logsData = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = tf.keras.callbacks.TensorBroad(log_dir = logsData, histogram_freq=1)



AttributeError: module 'keras.api._v2.keras.callbacks' has no attribute 'TensorBroad'

In [ ]:
history = model.fit(X_train, Y_train, epochs= 10, validation_data = (X_test,Y_test), verbose=1)

In [ ]:
model.save("Models/CatsandDogs.h5")

In [ ]:
class_data = ["cat","dog"]
saveModelData = load_model("Models/CatsandDogs.h5")

In [28]:
testImageData = cv2.imread("C:/Users/Anik/Downloads/cat.jpeg")
testImageDataFeature = ImageFeatureExtract(testImageData)/255.0
testImageDataFeature = np.array([testImageDataFeature])

NameError: name 'ImageFeatureExtract' is not defined

In [27]:
testImageDataFeature.shape

NameError: name 'testImageDataFeature' is not defined